Permutons for variance reduction on PageRank

In [771]:
import numpy as np
import random as rnd
import time
import matplotlib
from matplotlib import pyplot as plt
from numpy import array, zeros, diag, diagflat, dot
from math import sqrt
import pprint
import scipy
import scipy.linalg
import os
os.chdir('/homes/ir337/Documents/learnable_qmc')
from tqdm import tqdm
from utils import *
from rf_construction import *
from extra_funcs import *
from scipy import stats
id_norm = stats.norm()
from length_samplers import *
import itertools
from scipy.optimize import linear_sum_assignment

In [772]:
As = np.load('adj_matrices.npy',allow_pickle=True) #load the adjacency matrices
p_halt = 0.2
nb_trials = 4  #num of walkers out of each node

nb_bins = 10        #hyperparameter for order of permutation to learn
nb_per_bin = 10     #number of samples to take in every permutation bin

variance_repeats = 1000     #number of repeats when comparing approximation quality

In [773]:
def get_groundtruth_stationary(A, p_halt):
    "find the exact PageRank vector -- stationary dist of the Markov chain"
    nb_vertices = len(A)
    degrees = np.sum(A,axis=1)
    P = (A.T / degrees).T
    pi = p_halt/nb_vertices * np.sum(np.linalg.inv(np.eye(nb_vertices) - (1-p_halt) * P), axis=0) 
    return pi

def get_walk(adj_lists,base_vertex, length):
    "get walk endpoints from lengths we've sampled"
    
    current_vertex = base_vertex
    for _ in range(length):
      
        rnd_index = int(rnd.uniform(0,1) * len(adj_lists[current_vertex]))
        current_vertex = adj_lists[current_vertex][rnd_index]

    return current_vertex

In [774]:
A = As[1]
nb_vertices = np.shape(A)[0]
adj_lists, weight_lists, visits_list = adj_matrix_to_lists(A, qmc=True)
pi = get_groundtruth_stationary(A, p_halt)

In [775]:
def sample_projections(nb_bins, nb_per_bin,p_halt):
    "generate projections for every bin encoding where walker ends up given starting node and respective length. Gives nb_bins * nb_nodes * nb_nodes matrix matrix"

    unif_bins = np.linspace(0,1 - 1/(nb_bins * nb_per_bin),nb_bins * nb_per_bin)
    lengths = np.asarray(np.floor(np.log(1 - unif_bins) / np.log(1 - p_halt)), dtype = int)

    features = []
    for bin in range(nb_bins):
        this_feature = np.zeros((nb_vertices, nb_vertices))
        for _ in range(nb_trials):
            for base_vertex in range(nb_vertices):
                for length in lengths[bin*nb_per_bin: (bin+1)*nb_per_bin]:
                    end_vertex = get_walk(adj_lists, base_vertex, length)
                    this_feature[base_vertex,end_vertex] += 1
        features.append(this_feature)

    return features

def get_cost_matrix(projections):
    "return a cost matrix given projections"

    nb_bins = len(projections)
    cost_matrix = np.zeros((nb_bins,nb_bins))
    for i in range(nb_bins):
        for j in range(nb_bins):
            cost_matrix[i,j] += np.sum(projections[i] * projections[j])
    
    return cost_matrix



In [776]:
projections = sample_projections(nb_bins,nb_per_bin,p_halt)        #compute projections
cost_matrix = get_cost_matrix(projections)      #assemble in cost matrix
row_indices, optimal_permutation = scipy.optimize.linear_sum_assignment(cost_matrix)        #solve linear optimisation

In [777]:
optimal_permutation     #the best choice of permuton for PageRank among class

array([2, 5, 0, 9, 8, 1, 7, 6, 4, 3])

In [778]:
#Sampling lengths to construct PageRank estimates

def sample_iid_lengths(p, nb):
    "pair of independent lengths"
    lengths = []
    for _ in range(nb):
        out = []
        for _ in range(2):
            counter = 0
            while np.random.random()>p:
                counter += 1
            out.append(counter)
        lengths.append(out)
    return lengths

def sample_antithetic_lengths(p, nb):
    "pair of antithetic lengths"
    lengths = []
    for _ in range(nb):

        terminated = np.zeros(2)
        counters = np.zeros(2)
        while np.sum(terminated) < 2:
            rv = np.random.random()
            rvs = [rv, np.mod(rv+0.5,1)]
            if terminated[0] == 0:
                terminated[0] = rvs[0] < p
                counters[0] += rvs[0] > p
            if terminated[1] == 0:
                terminated[1] = rvs[1] < p
                counters[1] += rvs[1] > p
        counters[0] = (counters[0])
        counters[1] = (counters[1])
        lengths.append([int(counters[0]),int(counters[1])])
    return lengths

def sample_permuton(permutation):
    "arbitrary n permuton, marginally uniformly distributed"
    n = len(permutation)   #length of permutation = number of elements in the grid
    u1 = np.random.random()
    box = int(np.floor(u1 * n))
    target = permutation[box]
    u2 = (target + np.random.random()) / n
    return (u1,u2)

def sample_permuton_lengths(p,nb,permutation):
    "get the corresponding permuton lengths"
    lengths = []
    for _ in range(nb):
        out = []
        rvs = sample_permuton(permutation)
        out.append( int(np.floor(np.log(1-rvs[0]) / np.log(1-p) )))
        out.append( int(np.floor(np.log(1-rvs[1]) / np.log(1-p) )))
        lengths.append(out)
    return lengths

def get_pred(lengths):
    "predict PageRank vector for graph given list of walk lengths"
    endpoints = np.zeros(nb_vertices)
    for base_vertex in range(nb_vertices):
        for length in lengths:
            endpoint = get_walk(adj_lists, base_vertex,length)
            endpoints[endpoint] += 1
    return endpoints / np.sum(endpoints)



In [779]:
# i.i.d.
error_log = []
for _ in tqdm(range(variance_repeats)):
    lengths = np.ndarray.flatten(np.asarray(sample_iid_lengths(p_halt,nb_trials)))
    pred = get_pred(lengths)
    error_log.append( np.sum((pred - pi)**2) / np.sum(pi**2))
print(np.mean(error_log))
print(np.std(error_log) / np.sqrt(variance_repeats))

 32%|███▏      | 318/1000 [00:00<00:00, 759.09it/s]

100%|██████████| 1000/1000 [00:01<00:00, 749.74it/s]

0.08316201478325891
0.0005924609439922973


In [780]:
# antithetic
error_log = []
for _ in tqdm(range(variance_repeats)):
    lengths = np.ndarray.flatten(np.asarray(sample_antithetic_lengths(p_halt,nb_trials)))
    pred = get_pred(lengths)
    error_log.append( np.sum((pred - pi)**2) / np.sum(pi**2))
print(np.mean(error_log))
print(np.std(error_log) / np.sqrt(variance_repeats))

100%|██████████| 1000/1000 [00:01<00:00, 713.66it/s]

0.08295445058558874
0.0005859935634669137


In [781]:
# Permuton
error_log = []
for _ in tqdm(range(variance_repeats)):
    lengths = np.ndarray.flatten(np.asarray(sample_permuton_lengths(p_halt,nb_trials,optimal_permutation)))
    pred = get_pred(lengths)
    error_log.append( np.sum((pred - pi)**2) / np.sum(pi**2))
print(np.mean(error_log))
print(np.std(error_log) / np.sqrt(variance_repeats))


  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:01<00:00, 777.59it/s]

0.08212933247676764
0.0005971939481975643
